In [25]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple, Optional

cwd = Path.cwd()

# Load the data
ll_df_cph = pd.read_csv(cwd / 'll_data' / 'll_cph.csv')
ll_df_aarhus = pd.read_csv(cwd / 'll_data' / 'll_aarhus.csv')
ll_df_odense = pd.read_csv(cwd / 'll_data' / 'll_odense.csv')
df_list = [ll_df_cph, ll_df_aarhus, ll_df_odense]

county_names = ['københavn', 'århus', 'odense']

/var/folders/xr/n973hf0s243g7hdvvvh5kzmr0000gn/T/ipykernel_17145/391221310.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  ll_df_cph = pd.read_csv(cwd / 'll_data' / 'll_cph.csv')


In [26]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional

def analyze_never_married_rates(df_list, county_names):
    """
    Calculates never-married rates using Hajnal's age-50 method and a dynamic method
    across all age groups for men and women in Danish census data.
    
    Args:
        df_list: List of DataFrames containing census data for each county
        county_names: List of county names corresponding to each DataFrame
    
    Returns:
        Tuple of four DataFrames containing results for:
        - Hajnal method for men
        - Hajnal method for women
        - Dynamic method for men
        - Dynamic method for women
    """
    # Define constants for Hajnal method age range
    HAJNAL_AGE_MIN, HAJNAL_AGE_MAX = 49, 54
    
    # Define age ranges for dynamic method analysis
    age_ranges = [(15, 24), (25, 34), (35, 44), (45, 54), (55, 64)]
    
    # Initialize result storage
    hajnal_men_results, hajnal_women_results = [], []
    dynamic_men_results, dynamic_women_results = [], []
    
    # Process each county
    for df, county in zip(df_list, county_names):
        # Clean and prepare data
        df_clean = df.copy()
        df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
        df_valid = df_clean.dropna(subset=['age', 'marital_status', 'sex'])
        
        # Process each census year
        for year in sorted(df_valid['event_year'].unique()):
            year_data = df_valid[df_valid['event_year'] == year]
            
            # Process separately for men and women
            for sex_value, sex_label in [('m', 'Male'), ('f', 'Female')]:
                sex_data = year_data[year_data['sex'] == sex_value]
                
                # HAJNAL METHOD: calculate proportion never married at age 49-54
                hajnal_age_group = sex_data[(sex_data['age'] >= HAJNAL_AGE_MIN) & 
                                           (sex_data['age'] <= HAJNAL_AGE_MAX)]
                
                if len(hajnal_age_group) > 0:
                    # Count never married (ugift) individuals
                    never_married_count = hajnal_age_group[
                        hajnal_age_group['marital_status'].str.contains('ugift', case=False, na=False)
                    ].shape[0]
                    
                    # Calculate proportion and store result
                    hajnal_rate = never_married_count / len(hajnal_age_group)
                    result = {
                        'County': county,
                        'Year': year,
                        'Sex': sex_label,
                        'Never_Married_Rate': hajnal_rate,
                        'Sample_Size': len(hajnal_age_group)
                    }
                    
                    if sex_value == 'm':
                        hajnal_men_results.append(result)
                    else:
                        hajnal_women_results.append(result)
                
                # DYNAMIC METHOD: calculate for each age range
                for age_min, age_max in age_ranges:
                    age_group = sex_data[(sex_data['age'] >= age_min) & 
                                        (sex_data['age'] <= age_max)]
                    
                    if len(age_group) > 0:
                        # Count never married (ugift) individuals
                        never_married_count = age_group[
                            age_group['marital_status'].str.contains('ugift', case=False, na=False)
                        ].shape[0]
                        
                        # Calculate proportion and store result
                        dynamic_rate = never_married_count / len(age_group)
                        result = {
                            'County': county,
                            'Year': year,
                            'Sex': sex_label,
                            'Age_Group': f"{age_min}-{age_max}",
                            'Never_Married_Rate': dynamic_rate,
                            'Sample_Size': len(age_group)
                        }
                        
                        if sex_value == 'm':
                            dynamic_men_results.append(result)
                        else:
                            dynamic_women_results.append(result)
    
    # Convert results to DataFrames
    hajnal_men_df = pd.DataFrame(hajnal_men_results)
    hajnal_women_df = pd.DataFrame(hajnal_women_results)
    dynamic_men_df = pd.DataFrame(dynamic_men_results)
    dynamic_women_df = pd.DataFrame(dynamic_women_results)
    
    return hajnal_men_df, hajnal_women_df, dynamic_men_df, dynamic_women_df

def calculate_synthetic_cohort_estimates(dynamic_df, year):
    """
    Calculates synthetic cohort never-married estimates from age-specific rates
    for a given year.
    
    Args:
        dynamic_df: DataFrame with dynamic method results
        year: Year to calculate synthetic cohort for
    
    Returns:
        DataFrame with synthetic cohort estimates
    """
    # Filter for the specified year
    year_data = dynamic_df[dynamic_df['Year'] == year]
    results = []
    
    # Calculate for each county and sex combination
    for county in year_data['County'].unique():
        for sex in year_data['Sex'].unique():
            county_sex_data = year_data[
                (year_data['County'] == county) & 
                (year_data['Sex'] == sex)
            ]
            
            if len(county_sex_data) == 0:
                continue
            
            # Calculate average never-married rate across age groups
            # This is a simplified synthetic cohort approach
            synthetic_rate = county_sex_data['Never_Married_Rate'].mean()
            
            # Also calculate rate from oldest age group as an alternative estimate
            county_sex_data = county_sex_data.sort_values(by='Age_Group')
            oldest_group_rate = county_sex_data.iloc[-1]['Never_Married_Rate']
            
            # Store results
            results.append({
                'County': county,
                'Year': year,
                'Sex': sex,
                'Synthetic_Never_Married_Rate': synthetic_rate,
                'Oldest_Group_Rate': oldest_group_rate
            })
    
    return pd.DataFrame(results)

def process_marriage_data(df_list, county_names):
    """
    Main function to process marriage data and generate all analysis results.
    
    Args:
        df_list: List of DataFrames containing census data
        county_names: List of county names corresponding to each DataFrame
    
    Returns:
        Dictionary containing all processed results
    """
    # Step 1: Calculate never-married rates using both methods
    hajnal_men_df, hajnal_women_df, dynamic_men_df, dynamic_women_df = analyze_never_married_rates(
        df_list, county_names
    )
    
    # Step 2: Calculate synthetic cohort estimates for each year
    synthetic_results = []
    for year in sorted(set(dynamic_men_df['Year'].unique()) | set(dynamic_women_df['Year'].unique())):
        # Combine men and women for this year
        year_dynamic = pd.concat([
            dynamic_men_df[dynamic_men_df['Year'] == year],
            dynamic_women_df[dynamic_women_df['Year'] == year]
        ])
        
        # Calculate synthetic cohort estimates
        synthetic_year = calculate_synthetic_cohort_estimates(year_dynamic, year)
        if not synthetic_year.empty:
            synthetic_results.append(synthetic_year)
    
    # Combine all synthetic results
    synthetic_df = pd.concat(synthetic_results, ignore_index=True) if synthetic_results else pd.DataFrame()
    
    # Format percentages for display
    for df in [hajnal_men_df, hajnal_women_df]:
        if not df.empty:
            df['Rate_Percent'] = (df['Never_Married_Rate'] * 100).round(1)
    
    if not synthetic_df.empty:
        synthetic_df['Rate_Percent'] = (synthetic_df['Synthetic_Never_Married_Rate'] * 100).round(1)
    
    # Return all results in a dictionary
    return {
        'hajnal_men': hajnal_men_df,
        'hajnal_women': hajnal_women_df,
        'dynamic_men': dynamic_men_df,
        'dynamic_women': dynamic_women_df,
        'synthetic': synthetic_df
    }

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple, Optional
from matplotlib.backends.backend_pdf import PdfPages

def visualize_never_married_rates(results, copenhagen_missing_year=1834, min_sample_size=10):
    """
    Creates visualizations for never-married rates analysis with separate plots for each county,
    consistent scaling, year-specific x-axis ticks, and visual distinction between methods.
    
    Args:
        results: Dictionary containing analysis results from process_marriage_data
        copenhagen_missing_year: Year to mark with vertical line for København (default: 1834)
        min_sample_size: Minimum sample size to include in analysis (default: 10)
    
    Returns:
        Dictionary with paths to generated visualizations
    """
    # Set visualization style
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_context("talk")
    
    # Extract data from results
    hajnal_men_df = results['hajnal_men']
    hajnal_women_df = results['hajnal_women']
    dynamic_men_df = results['dynamic_men']
    dynamic_women_df = results['dynamic_women']
    synthetic_df = results['synthetic']
    
    # Filter for minimum sample size
    hajnal_men = hajnal_men_df[hajnal_men_df['Sample_Size'] >= min_sample_size]
    hajnal_women = hajnal_women_df[hajnal_women_df['Sample_Size'] >= min_sample_size]
    
    # Get all counties
    counties = sorted(set(
        hajnal_men['County'].unique().tolist() + 
        hajnal_women['County'].unique().tolist() + 
        synthetic_df['County'].unique().tolist()
    ))
    
    # Find global max rates for consistent scaling
    max_rate = max(
        hajnal_men['Never_Married_Rate'].max() if not hajnal_men.empty else 0,
        hajnal_women['Never_Married_Rate'].max() if not hajnal_women.empty else 0,
        synthetic_df['Synthetic_Never_Married_Rate'].max() if not synthetic_df.empty else 0
    ) * 100 * 1.1  # Add 10% margin for better visualization
    
    # Define color scheme for methods and genders
    colors = {
        'hajnal_men': '#1f77b4',    # Blue
        'hajnal_women': '#d62728',  # Red
        'dynamic_men': '#72b7f2',   # Lighter blue
        'dynamic_women': '#ff9896'  # Lighter red
    }
    
    output_files = {}
    
    # Create Hajnal method visualizations for each county
    hajnal_figures = []
    for county in counties:
        # Create figure for this county
        fig, ax = plt.subplots(figsize=(10, 7))
        
        # Filter data for this county
        county_men = hajnal_men[hajnal_men['County'] == county]
        county_women = hajnal_women[hajnal_women['County'] == county]
        
        # Get years with data for this county (for x-axis ticks)
        county_years = sorted(set(
            county_men['Year'].unique().tolist() + 
            county_women['Year'].unique().tolist()
        ))
        
        # Plot men's data
        if not county_men.empty:
            ax.plot(
                county_men['Year'], 
                county_men['Never_Married_Rate'] * 100,
                marker='o',
                linestyle='-',
                color=colors['hajnal_men'],
                linewidth=2,
                label="Men"
            )
        
        # Plot women's data
        if not county_women.empty:
            ax.plot(
                county_women['Year'], 
                county_women['Never_Married_Rate'] * 100,
                marker='s',
                linestyle='-',
                color=colors['hajnal_women'],
                linewidth=2,
                label="Women"
            )
        
        # Mark København missing data with vertical line
        if county.lower() == 'københavn' and copenhagen_missing_year in county_years:
            ax.axvline(x=copenhagen_missing_year, color='red', linestyle='--', linewidth=2)
            ax.text(
                copenhagen_missing_year + 0.5, 
                max_rate * 0.95,
                "A' (Missing Data)",
                color='red',
                fontweight='bold',
                ha='left',
                va='top'
            )
        
        # Configure axis and labels
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Never-Married Rate (%)', fontsize=12)
        ax.set_title(f'Never-Married Rates in {county.capitalize()} (Hajnal Method)', fontsize=14)
        ax.set_ylim(0, max_rate)  # Consistent y-axis scaling
        ax.set_xticks(county_years)  # Only years with data
        ax.set_xticklabels(county_years, rotation=45)
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best', fontsize=10)
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        
        plt.tight_layout()
        
        # Save figure
        filename = f'hajnal_never_married_{county}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        hajnal_figures.append(fig)
        output_files[f'hajnal_{county}'] = filename
        
        # Do not close figure yet - we'll close all at the end
    
    # Create method comparison visualizations for each county
    comparison_figures = []
    for county in counties:
        # Create figure with two rows (men and women)
        fig, axes = plt.subplots(2, 1, figsize=(12, 14), sharex=True)
        ax_men, ax_women = axes
        
        # Filter data for this county
        county_hajnal_men = hajnal_men[hajnal_men['County'] == county]
        county_synth_men = synthetic_df[(synthetic_df['County'] == county) & 
                                       (synthetic_df['Sex'] == 'Male')]
        
        county_hajnal_women = hajnal_women[hajnal_women['County'] == county]
        county_synth_women = synthetic_df[(synthetic_df['County'] == county) & 
                                         (synthetic_df['Sex'] == 'Female')]
        
        # Get years with data for this county
        men_years = sorted(set(
            county_hajnal_men['Year'].unique().tolist() + 
            county_synth_men['Year'].unique().tolist()
        ))
        
        women_years = sorted(set(
            county_hajnal_women['Year'].unique().tolist() + 
            county_synth_women['Year'].unique().tolist()
        ))
        
        all_years = sorted(set(men_years + women_years))
        
        # Plot men's data
        if not county_hajnal_men.empty:
            ax_men.plot(
                county_hajnal_men['Year'], 
                county_hajnal_men['Never_Married_Rate'] * 100,
                marker='o',
                linestyle='-',
                color=colors['hajnal_men'],
                linewidth=2,
                label="Hajnal Method"
            )
        
        if not county_synth_men.empty:
            ax_men.plot(
                county_synth_men['Year'], 
                county_synth_men['Synthetic_Never_Married_Rate'] * 100,
                marker='o',
                linestyle='--',  # Dashed line for dynamic method
                color=colors['dynamic_men'],  # Lighter shade for dynamic method
                linewidth=2,
                label="Dynamic Method"
            )
        
        # Plot women's data
        if not county_hajnal_women.empty:
            ax_women.plot(
                county_hajnal_women['Year'], 
                county_hajnal_women['Never_Married_Rate'] * 100,
                marker='s',
                linestyle='-',
                color=colors['hajnal_women'],
                linewidth=2,
                label="Hajnal Method"
            )
        
        if not county_synth_women.empty:
            ax_women.plot(
                county_synth_women['Year'], 
                county_synth_women['Synthetic_Never_Married_Rate'] * 100,
                marker='s',
                linestyle='--',  # Dashed line for dynamic method
                color=colors['dynamic_women'],  # Lighter shade for dynamic method
                linewidth=2,
                label="Dynamic Method"
            )
        
        # Configure axes
        max_rate_men = max(
            county_hajnal_men['Never_Married_Rate'].max() if not county_hajnal_men.empty else 0,
            county_synth_men['Synthetic_Never_Married_Rate'].max() if not county_synth_men.empty else 0
        ) * 100 * 1.1
        
        max_rate_women = max(
            county_hajnal_women['Never_Married_Rate'].max() if not county_hajnal_women.empty else 0,
            county_synth_women['Synthetic_Never_Married_Rate'].max() if not county_synth_women.empty else 0
        ) * 100 * 1.1
        
        # Use the same max rate for both to maintain consistency
        common_max = max(max_rate_men, max_rate_women, max_rate)
        
        ax_men.set_ylabel('Never-Married Rate (%)', fontsize=12)
        ax_men.set_title(f'Never-Married Rates for Men in {county.capitalize()}', fontsize=14)
        ax_men.grid(True, alpha=0.3)
        ax_men.legend(loc='best', fontsize=10)
        ax_men.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        ax_men.set_ylim(0, common_max)
        ax_men.set_xticks(all_years)
        
        ax_women.set_xlabel('Year', fontsize=12)
        ax_women.set_ylabel('Never-Married Rate (%)', fontsize=12)
        ax_women.set_title(f'Never-Married Rates for Women in {county.capitalize()}', fontsize=14)
        ax_women.grid(True, alpha=0.3)
        ax_women.legend(loc='best', fontsize=10)
        ax_women.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        ax_women.set_ylim(0, common_max)
        ax_women.set_xticks(all_years)
        ax_women.set_xticklabels(all_years, rotation=45)
        
        # Mark Copenhagen missing data
        if county.lower() == 'københavn' and copenhagen_missing_year in all_years:
            for ax in [ax_men, ax_women]:
                ax.axvline(x=copenhagen_missing_year, color='red', linestyle='--', linewidth=2)
                ax.text(
                    copenhagen_missing_year + 0.5, 
                    ax.get_ylim()[1] * 0.95,
                    "A' (Missing Data)",
                    color='red',
                    fontweight='bold',
                    ha='left',
                    va='top'
                )
        
        plt.tight_layout()
        
        # Save figure
        filename = f'method_comparison_{county}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        comparison_figures.append(fig)
        output_files[f'comparison_{county}'] = filename
    
    # Create dashboard with county-specific panels
    dashboard_fig = create_dashboard(
        results,
        counties=counties,
        copenhagen_missing_year=copenhagen_missing_year
    )
    dashboard_path = 'never_married_dashboard.png'
    dashboard_fig.savefig(dashboard_path, dpi=300, bbox_inches='tight')
    output_files['dashboard'] = dashboard_path
    
    # Create comprehensive tables of never-married rates
    table_path = create_never_married_tables(
        hajnal_men_df,
        hajnal_women_df,
        synthetic_df
    )
    output_files['tables'] = table_path
    
    # Close all figures
    for fig in hajnal_figures + comparison_figures + [dashboard_fig]:
        plt.close(fig)
    
    return output_files

def create_dashboard(results, counties, copenhagen_missing_year=1834):
    """
    Creates a comprehensive dashboard visualization of never-marriage patterns
    with separate panels for each county.
    
    Args:
        results: Dictionary containing analysis results
        counties: List of counties to include
        copenhagen_missing_year: Year with missing data for Copenhagen
        
    Returns:
        Matplotlib figure object
    """
    from matplotlib.gridspec import GridSpec
    
    # Extract data
    hajnal_men_df = results['hajnal_men']
    hajnal_women_df = results['hajnal_women']
    dynamic_men_df = results['dynamic_men']
    dynamic_women_df = results['dynamic_women']
    synthetic_df = results['synthetic']
    
    # Set up figure with title + 2 rows per county
    rows = 2 + 2 * len(counties)
    fig = plt.figure(figsize=(22, 8 + 10 * len(counties)))
    gs = GridSpec(rows, 2, figure=fig)
    
    # Title area (spans first two rows and both columns)
    title_ax = fig.add_subplot(gs[0:2, :])
    title_ax.axis('off')
    
    # Add title and description
    title_ax.text(0.5, 0.8, 'Never-Marriage Patterns in 19th Century Denmark',
                 fontsize=24, ha='center', weight='bold')
    
    description = (
        "This dashboard compares never-marriage rates across Danish counties using two methodologies:\n"
        "• Hajnal's Age-50 Method: Examining unmarried individuals aged 49-54 years\n"
        "• Dynamic Method: Analyzing marriage patterns across all age groups\n\n"
        "The analysis reveals regional and gender differences in marriage patterns throughout the 19th century."
    )
    
    title_ax.text(0.5, 0.4, description, fontsize=14, ha='center', va='top', 
                 linespacing=1.5, transform=title_ax.transAxes)
    
    # Find max rates for consistent scaling
    max_rate = max(
        hajnal_men_df['Never_Married_Rate'].max() if not hajnal_men_df.empty else 0,
        hajnal_women_df['Never_Married_Rate'].max() if not hajnal_women_df.empty else 0,
        synthetic_df['Synthetic_Never_Married_Rate'].max() if not synthetic_df.empty else 0
    ) * 100 * 1.1
    
    # Colors for different methods and genders
    colors = {
        'hajnal_men': '#1f77b4',    # Blue
        'hajnal_women': '#d62728',  # Red
        'dynamic_men': '#72b7f2',   # Light blue
        'dynamic_women': '#ff9896'  # Light red
    }
    
    # Process each county
    for i, county in enumerate(counties):
        # Row offset (title takes 2 rows, then each county takes 2 rows)
        row_offset = 2 + i * 2
        
        # Men subplot (left column) and women subplot (right column)
        men_ax = fig.add_subplot(gs[row_offset, 0])
        women_ax = fig.add_subplot(gs[row_offset, 1])
        
        # Age distribution subplot (row below, spans both columns)
        age_ax = fig.add_subplot(gs[row_offset + 1, :])
        
        # Get all years with data for this county
        county_years = sorted(set(
            hajnal_men_df[hajnal_men_df['County'] == county]['Year'].unique().tolist() + 
            hajnal_women_df[hajnal_women_df['County'] == county]['Year'].unique().tolist() + 
            synthetic_df[synthetic_df['County'] == county]['Year'].unique().tolist()
        ))
        
        # Filter data for this county
        county_hajnal_men = hajnal_men_df[hajnal_men_df['County'] == county]
        county_synth_men = synthetic_df[(synthetic_df['County'] == county) & 
                                        (synthetic_df['Sex'] == 'Male')]
        
        county_hajnal_women = hajnal_women_df[hajnal_women_df['County'] == county]
        county_synth_women = synthetic_df[(synthetic_df['County'] == county) & 
                                          (synthetic_df['Sex'] == 'Female')]
        
        # Plot men's data
        if not county_hajnal_men.empty:
            men_ax.plot(
                county_hajnal_men['Year'], 
                county_hajnal_men['Never_Married_Rate'] * 100,
                marker='o', linestyle='-', color=colors['hajnal_men'],
                linewidth=2, label="Hajnal Method"
            )
        
        if not county_synth_men.empty:
            men_ax.plot(
                county_synth_men['Year'], 
                county_synth_men['Synthetic_Never_Married_Rate'] * 100,
                marker='o', linestyle='--', color=colors['dynamic_men'],
                linewidth=2, label="Dynamic Method"
            )
        
        # Plot women's data
        if not county_hajnal_women.empty:
            women_ax.plot(
                county_hajnal_women['Year'], 
                county_hajnal_women['Never_Married_Rate'] * 100,
                marker='s', linestyle='-', color=colors['hajnal_women'],
                linewidth=2, label="Hajnal Method"
            )
        
        if not county_synth_women.empty:
            women_ax.plot(
                county_synth_women['Year'], 
                county_synth_women['Synthetic_Never_Married_Rate'] * 100,
                marker='s', linestyle='--', color=colors['dynamic_women'],
                linewidth=2, label="Dynamic Method"
            )
        
        # Mark Copenhagen missing data
        if county.lower() == 'københavn' and copenhagen_missing_year in county_years:
            for ax in [men_ax, women_ax]:
                ax.axvline(x=copenhagen_missing_year, color='red', linestyle='--', linewidth=2)
                ax.text(
                    copenhagen_missing_year + 0.5, 
                    max_rate * 0.95,
                    "A' (Missing Data)",
                    color='red', fontweight='bold', ha='left', va='top'
                )
        
        # Configure men and women plots
        for ax, title in zip([men_ax, women_ax], 
                           [f'Men in {county.capitalize()}', 
                            f'Women in {county.capitalize()}']):
            ax.set_xlabel('Year', fontsize=12)
            ax.set_ylabel('Never-Married Rate (%)', fontsize=12)
            ax.set_title(title, fontsize=14)
            ax.set_ylim(0, max_rate)
            ax.set_xticks(county_years)
            ax.set_xticklabels(county_years, rotation=45)
            ax.grid(True, alpha=0.3)
            ax.legend(loc='best', fontsize=10)
            ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        
        # Plot age distribution for this county (latest year)
        county_dynamic_men = dynamic_men_df[dynamic_men_df['County'] == county]
        county_dynamic_women = dynamic_women_df[dynamic_women_df['County'] == county]
        
        if not county_dynamic_men.empty or not county_dynamic_women.empty:
            # Get latest year with data
            dynamic_years = sorted(set(
                county_dynamic_men['Year'].unique().tolist() + 
                county_dynamic_women['Year'].unique().tolist()
            ))
            
            if dynamic_years:
                latest_year = max(dynamic_years)
                
                # Plot age patterns for latest year
                for df, gender, color in [
                    (county_dynamic_men[county_dynamic_men['Year'] == latest_year], 
                     'Men', colors['hajnal_men']),
                    (county_dynamic_women[county_dynamic_women['Year'] == latest_year], 
                     'Women', colors['hajnal_women'])
                ]:
                    if not df.empty:
                        # Extract age groups and convert to midpoints
                        age_groups = df['Age_Group'].tolist()
                        age_midpoints = []
                        for ag in age_groups:
                            min_age, max_age = map(int, ag.split('-'))
                            age_midpoints.append((min_age + max_age) / 2)
                        
                        # Sort by age
                        idx = np.argsort(age_midpoints)
                        sorted_midpoints = [age_midpoints[k] for k in idx]
                        rates = df['Never_Married_Rate'].values[idx] * 100
                        
                        # Plot age pattern
                        age_ax.plot(
                            sorted_midpoints, rates,
                            marker='o' if gender == 'Men' else 's',
                            linestyle='-', color=color,
                            label=f"{gender} ({latest_year})"
                        )
                
                # Configure age plot
                age_ax.set_xlabel('Age (midpoint of age group)', fontsize=12)
                age_ax.set_ylabel('Never-Married Rate (%)', fontsize=12)
                age_ax.set_title(f'Age-Specific Never-Married Rates in {county.capitalize()} ({latest_year})', 
                               fontsize=14)
                age_ax.grid(True, alpha=0.3)
                age_ax.legend(loc='best', fontsize=10)
                age_ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
                age_ax.set_ylim(0, 100)
    
    # Add summary box with key findings
    if not hajnal_men_df.empty and not hajnal_women_df.empty:
        # Create summary subplot in the last row
        summary_ax = fig.add_subplot(gs[rows-1, :])
        summary_ax.axis('off')
        
        # Find highest rates and gender gaps
        max_men_rate = hajnal_men_df.loc[hajnal_men_df['Never_Married_Rate'].idxmax()]
        max_women_rate = hajnal_women_df.loc[hajnal_women_df['Never_Married_Rate'].idxmax()]
        
        # Calculate gender gaps
        merged_data = pd.merge(
            hajnal_men_df[['County', 'Year', 'Never_Married_Rate']],
            hajnal_women_df[['County', 'Year', 'Never_Married_Rate']],
            on=['County', 'Year'],
            suffixes=('_men', '_women')
        )
        
        merged_data['gender_gap'] = merged_data['Never_Married_Rate_men'] - merged_data['Never_Married_Rate_women']
        max_gap = merged_data.loc[merged_data['gender_gap'].abs().idxmax()]
        
        # Create summary text
        findings = (
            f"Key Findings:\n"
            f"• Highest never-married rate for men: {max_men_rate['County']} ({max_men_rate['Year']}): {max_men_rate['Never_Married_Rate']*100:.1f}%\n"
            f"• Highest never-married rate for women: {max_women_rate['County']} ({max_women_rate['Year']}): {max_women_rate['Never_Married_Rate']*100:.1f}%\n"
            f"• Largest gender gap: {max_gap['County']} ({max_gap['Year']}): {abs(max_gap['gender_gap']*100):.1f}% "
            f"({'men' if max_gap['gender_gap'] > 0 else 'women'} more likely to remain unmarried)\n"
        )
        
        # Add visualization guide
        method_legend = (
            "Visualization Guide:\n"
            "• Solid lines with dark colors: Hajnal method (ages 49-54)\n"
            "• Dashed lines with lighter colors: Dynamic synthetic cohort method\n"
            f"• Red vertical line with 'A'' in København graphs: Missing data for year {copenhagen_missing_year}\n"
            "• All county graphs use the same scale for direct comparison"
        )
        
        # Add text box with findings
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
        summary_ax.text(0.5, 0.5, findings + "\n\n" + method_legend, 
                      transform=summary_ax.transAxes, fontsize=12,
                      verticalalignment='center', horizontalalignment='center', 
                      bbox=props, linespacing=1.5)
    
    # Adjust layout
    plt.tight_layout()
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    
    return fig

def create_never_married_tables(hajnal_men_df, hajnal_women_df, synthetic_df, output_path="never_married_rates.pdf"):
    """
    Creates comprehensive tables of never-marriage rates by county, year, method, and gender.
    
    Args:
        hajnal_men_df: DataFrame with Hajnal method results for men
        hajnal_women_df: DataFrame with Hajnal method results for women
        synthetic_df: DataFrame with synthetic cohort estimates
        output_path: Path to save the PDF table
        
    Returns:
        Path to the saved table file
    """
    # Create a combined dataset
    results = []
    
    # Process Hajnal data for men
    for _, row in hajnal_men_df.iterrows():
        results.append({
            'County': row['County'],
            'Year': row['Year'],
            'Gender': 'Male',
            'Method': 'Hajnal',
            'Never_Married_Rate': row['Never_Married_Rate'] * 100,
            'Sample_Size': row['Sample_Size']
        })
    
    # Process Hajnal data for women
    for _, row in hajnal_women_df.iterrows():
        results.append({
            'County': row['County'],
            'Year': row['Year'],
            'Gender': 'Female',
            'Method': 'Hajnal',
            'Never_Married_Rate': row['Never_Married_Rate'] * 100,
            'Sample_Size': row['Sample_Size']
        })
    
    # Process Synthetic data
    for _, row in synthetic_df.iterrows():
        results.append({
            'County': row['County'],
            'Year': row['Year'],
            'Gender': row['Sex'],
            'Method': 'Dynamic',
            'Never_Married_Rate': row['Synthetic_Never_Married_Rate'] * 100,
            'Sample_Size': 'N/A'  # Synthetic method doesn't have direct sample sizes
        })
    
    # Convert to DataFrame and sort
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(['County', 'Year', 'Gender', 'Method'])
    
    # Format rates to have consistent decimal places
    results_df['Never_Married_Rate'] = results_df['Never_Married_Rate'].round(1)
    
    # Create a pivot table for easier reading
    pivot_df = results_df.pivot_table(
        index=['County', 'Year'],
        columns=['Gender', 'Method'],
        values='Never_Married_Rate',
        aggfunc='first'
    ).reset_index()
    
    # Sort by county and year
    pivot_df = pivot_df.sort_values(['County', 'Year'])
    
    # Save as CSV in case PDF generation fails
    csv_path = output_path.replace('.pdf', '.csv')
    pivot_df.to_csv(csv_path, index=False)
    
    # Try to create PDF table
    try:
        # Create PDF with tables
        with PdfPages(output_path) as pdf:
            # Create a title page
            plt.figure(figsize=(8.5, 11))
            plt.axis('off')
            plt.text(0.5, 0.5, "Never-Married Rates in 19th Century Denmark\nBy County, Year, Gender, and Method",
                   ha='center', va='center', fontsize=16)
            pdf.savefig()
            plt.close()
            
            # Create a table for each county
            for county in pivot_df['County'].unique():
                county_data = pivot_df[pivot_df['County'] == county]
                
                # Remove county column since it's redundant
                county_data = county_data.drop(columns=['County'])
                
                fig, ax = plt.subplots(figsize=(8.5, 11))
                ax.axis('off')
                
                # Add county as title
                ax.text(0.5, 0.95, f"Never-Married Rates in {county.capitalize()}",
                       ha='center', va='top', fontsize=14, fontweight='bold')
                
                # Format the header to separate gender and method
                header = []
                for col in county_data.columns:
                    if isinstance(col, tuple):
                        header.append(f"{col[0]}\n({col[1]})")
                    else:
                        header.append(str(col))
                
                # Create the table
                table = ax.table(
                    cellText=county_data.values.tolist(),
                    colLabels=header,
                    loc='center',
                    cellLoc='center'
                )
                
                # Style the table
                table.auto_set_font_size(False)
                table.set_fontsize(9)
                table.scale(1.2, 1.5)
                
                # Style header and add percentage signs
                for (i, j), cell in table.get_celld().items():
                    if i == 0:  # Header row
                        cell.set_text_props(fontweight='bold')
                        cell.set_facecolor('#D3D3D3')
                    
                    # Add % to rate values
                    if i > 0 and j > 0:  # Data cells (not year)
                        try:
                            value = cell.get_text().get_text()
                            if value != 'nan':
                                cell.get_text().set_text(f"{value}%")
                        except:
                            pass
                
                pdf.savefig(fig, bbox_inches='tight')
                plt.close()
        
        return output_path
    except Exception as e:
        print(f"Error creating PDF: {e}. Data saved as CSV instead.")
        return csv_path

In [28]:
def run_never_married_analysis(df_list, county_names, copenhagen_missing_year=1834):
    """
    Run the complete never-marriage rate analysis pipeline from data preparation 
    through visualization and table generation.
    
    Args:
        df_list: List of DataFrames containing census data for each county
        county_names: List of county names corresponding to each DataFrame
        copenhagen_missing_year: Year to mark with vertical line for København (default: 1834)
    
    Returns:
        Dictionary containing analysis results and paths to generated visualizations
    """
    print("Starting analysis of never-marriage rates in 19th century Denmark...")
    
    # Step 1: Process and analyze data
    print("Processing census data and calculating never-married rates...")
    results = process_marriage_data(df_list, county_names)
    
    # Unpack results for easier access
    hajnal_men_df = results['hajnal_men']
    hajnal_women_df = results['hajnal_women']
    dynamic_men_df = results['dynamic_men']
    dynamic_women_df = results['dynamic_women']
    synthetic_df = results['synthetic']
    
    # Print summary statistics for each county
    for county in county_names:
        print(f"\n{county.capitalize()} County:")
        print("-" * 40)
        
        # Print Hajnal method results for men
        county_men = hajnal_men_df[hajnal_men_df['County'] == county].sort_values('Year')
        if not county_men.empty:
            print("Men's never-married rates (Hajnal method):")
            for _, row in county_men.iterrows():
                print(f"  {row['Year']}: {row['Rate_Percent']}% (n={row['Sample_Size']})")
        
        # Print Hajnal method results for women
        county_women = hajnal_women_df[hajnal_women_df['County'] == county].sort_values('Year')
        if not county_women.empty:
            print("\nWomen's never-married rates (Hajnal method):")
            for _, row in county_women.iterrows():
                print(f"  {row['Year']}: {row['Rate_Percent']}% (n={row['Sample_Size']})")
    
    # Step 2: Create visualizations
    print("\nGenerating visualizations...")
    
    # Get all counties for consistent visualization
    counties = sorted(set(
        hajnal_men_df['County'].unique().tolist() + 
        hajnal_women_df['County'].unique().tolist() + 
        synthetic_df['County'].unique().tolist()
    ))
    
    output_files = {}
    
    # Create Hajnal method visualizations for each county
    print("Creating county-specific Hajnal method plots...")
    for county in counties:
        fig, ax = plt.subplots(figsize=(10, 7))
        
        # Filter data for this county
        county_men = hajnal_men_df[hajnal_men_df['County'] == county]
        county_women = hajnal_women_df[hajnal_women_df['County'] == county]
        
        # Get years with data for this county (for x-axis ticks)
        county_years = sorted(set(
            county_men['Year'].unique().tolist() + 
            county_women['Year'].unique().tolist()
        ))
        
        # Find max rate for consistent y-axis scaling
        max_rate = max(
            hajnal_men_df['Never_Married_Rate'].max() if not hajnal_men_df.empty else 0,
            hajnal_women_df['Never_Married_Rate'].max() if not hajnal_women_df.empty else 0
        ) * 100 * 1.1  # Add 10% margin
        
        # Plot men's data
        if not county_men.empty:
            ax.plot(
                county_men['Year'], 
                county_men['Never_Married_Rate'] * 100,
                marker='o',
                linestyle='-',
                color='#1f77b4',  # Blue
                linewidth=2,
                label="Men"
            )
        
        # Plot women's data
        if not county_women.empty:
            ax.plot(
                county_women['Year'], 
                county_women['Never_Married_Rate'] * 100,
                marker='s',
                linestyle='-',
                color='#d62728',  # Red
                linewidth=2,
                label="Women"
            )
        
        # Mark København missing data with vertical line
        if county.lower() == 'københavn' and copenhagen_missing_year in county_years:
            ax.axvline(x=copenhagen_missing_year, color='red', linestyle='--', linewidth=2)
            ax.text(
                copenhagen_missing_year + 0.5, 
                max_rate * 0.95,
                "A' (Missing Data)",
                color='red',
                fontweight='bold',
                ha='left',
                va='top'
            )
        
        # Configure axis and labels
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Never-Married Rate (%)', fontsize=12)
        ax.set_title(f'Never-Married Rates in {county.capitalize()} (Hajnal Method)', fontsize=14)
        ax.set_ylim(0, max_rate)  # Consistent y-axis scaling
        ax.set_xticks(county_years)  # Only years with data
        ax.set_xticklabels(county_years, rotation=45)
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best', fontsize=10)
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        
        plt.tight_layout()
        
        # Save figure
        filename = f'hajnal_never_married_{county}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        output_files[f'hajnal_{county}'] = filename
        plt.close(fig)
    
    # Create method comparison visualizations for each county
    print("Creating county-specific method comparison plots...")
    for county in counties:
        # Create figure with two rows (men and women)
        fig, axes = plt.subplots(2, 1, figsize=(12, 14), sharex=True)
        ax_men, ax_women = axes
        
        # Filter data for this county
        county_hajnal_men = hajnal_men_df[hajnal_men_df['County'] == county]
        county_synth_men = synthetic_df[(synthetic_df['County'] == county) & 
                                       (synthetic_df['Sex'] == 'Male')]
        
        county_hajnal_women = hajnal_women_df[hajnal_women_df['County'] == county]
        county_synth_women = synthetic_df[(synthetic_df['County'] == county) & 
                                         (synthetic_df['Sex'] == 'Female')]
        
        # Get years with data for this county
        men_years = sorted(set(
            county_hajnal_men['Year'].unique().tolist() + 
            county_synth_men['Year'].unique().tolist()
        ))
        
        women_years = sorted(set(
            county_hajnal_women['Year'].unique().tolist() + 
            county_synth_women['Year'].unique().tolist()
        ))
        
        all_years = sorted(set(men_years + women_years))
        
        # Find max rates for each gender for consistent scaling
        max_rate_men = max(
            county_hajnal_men['Never_Married_Rate'].max() if not county_hajnal_men.empty else 0,
            county_synth_men['Synthetic_Never_Married_Rate'].max() if not county_synth_men.empty else 0
        ) * 100 * 1.1
        
        max_rate_women = max(
            county_hajnal_women['Never_Married_Rate'].max() if not county_hajnal_women.empty else 0,
            county_synth_women['Synthetic_Never_Married_Rate'].max() if not county_synth_women.empty else 0
        ) * 100 * 1.1
        
        # Use the same max rate for both to maintain consistency
        common_max = max(max_rate_men, max_rate_women)
        
        # Define colors for methods - darker for Hajnal, lighter for Dynamic
        colors = {
            'hajnal_men': '#1f77b4',    # Blue
            'hajnal_women': '#d62728',  # Red
            'dynamic_men': '#72b7f2',   # Lighter blue
            'dynamic_women': '#ff9896'  # Lighter red
        }
        
        # Plot men's data
        if not county_hajnal_men.empty:
            ax_men.plot(
                county_hajnal_men['Year'], 
                county_hajnal_men['Never_Married_Rate'] * 100,
                marker='o',
                linestyle='-',
                color=colors['hajnal_men'],
                linewidth=2,
                label="Hajnal Method"
            )
        
        if not county_synth_men.empty:
            ax_men.plot(
                county_synth_men['Year'], 
                county_synth_men['Synthetic_Never_Married_Rate'] * 100,
                marker='o',
                linestyle='--',  # Dashed line for dynamic method
                color=colors['dynamic_men'],  # Lighter shade for dynamic method
                linewidth=2,
                label="Dynamic Method"
            )
        
        # Plot women's data
        if not county_hajnal_women.empty:
            ax_women.plot(
                county_hajnal_women['Year'], 
                county_hajnal_women['Never_Married_Rate'] * 100,
                marker='s',
                linestyle='-',
                color=colors['hajnal_women'],
                linewidth=2,
                label="Hajnal Method"
            )
        
        if not county_synth_women.empty:
            ax_women.plot(
                county_synth_women['Year'], 
                county_synth_women['Synthetic_Never_Married_Rate'] * 100,
                marker='s',
                linestyle='--',  # Dashed line for dynamic method
                color=colors['dynamic_women'],  # Lighter shade for dynamic method
                linewidth=2,
                label="Dynamic Method"
            )
        
        # Configure axes
        ax_men.set_ylabel('Never-Married Rate (%)', fontsize=12)
        ax_men.set_title(f'Never-Married Rates for Men in {county.capitalize()}', fontsize=14)
        ax_men.grid(True, alpha=0.3)
        ax_men.legend(loc='best', fontsize=10)
        ax_men.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        ax_men.set_ylim(0, common_max)
        ax_men.set_xticks(all_years)
        
        ax_women.set_xlabel('Year', fontsize=12)
        ax_women.set_ylabel('Never-Married Rate (%)', fontsize=12)
        ax_women.set_title(f'Never-Married Rates for Women in {county.capitalize()}', fontsize=14)
        ax_women.grid(True, alpha=0.3)
        ax_women.legend(loc='best', fontsize=10)
        ax_women.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        ax_women.set_ylim(0, common_max)
        ax_women.set_xticks(all_years)
        ax_women.set_xticklabels(all_years, rotation=45)
        
        # Mark Copenhagen missing data
        if county.lower() == 'københavn' and copenhagen_missing_year in all_years:
            for ax in [ax_men, ax_women]:
                ax.axvline(x=copenhagen_missing_year, color='red', linestyle='--', linewidth=2)
                ax.text(
                    copenhagen_missing_year + 0.5, 
                    ax.get_ylim()[1] * 0.95,
                    "A' (Missing Data)",
                    color='red',
                    fontweight='bold',
                    ha='left',
                    va='top'
                )
        
        plt.tight_layout()
        
        # Save figure
        filename = f'method_comparison_{county}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        output_files[f'comparison_{county}'] = filename
        plt.close(fig)
    
    # Create comprehensive dashboard
    print("Creating comprehensive dashboard...")
    
    # Set up figure for dashboard with title + 2 rows per county + summary
    rows = 2 + 2 * len(counties) + 1
    dashboard_fig = plt.figure(figsize=(22, 8 + 10 * len(counties)))
    gs = GridSpec(rows, 2, figure=dashboard_fig)
    
    # Title area (spans first two rows and both columns)
    title_ax = dashboard_fig.add_subplot(gs[0:2, :])
    title_ax.axis('off')
    
    # Add title and description
    title_ax.text(0.5, 0.8, 'Never-Marriage Patterns in 19th Century Denmark',
                 fontsize=24, ha='center', weight='bold')
    
    description = (
        "This dashboard compares never-marriage rates across Danish counties using two methodologies:\n"
        "• Hajnal's Age-50 Method: Examining unmarried individuals aged 49-54 years\n"
        "• Dynamic Method: Analyzing marriage patterns across all age groups\n\n"
        "The analysis reveals regional and gender differences in marriage patterns throughout the 19th century."
    )
    
    title_ax.text(0.5, 0.4, description, fontsize=14, ha='center', va='top', 
                 linespacing=1.5, transform=title_ax.transAxes)
    
    # Find max rate for consistent scaling across all panels
    max_rate = max(
        hajnal_men_df['Never_Married_Rate'].max() if not hajnal_men_df.empty else 0,
        hajnal_women_df['Never_Married_Rate'].max() if not hajnal_women_df.empty else 0,
        synthetic_df['Synthetic_Never_Married_Rate'].max() if not synthetic_df.empty else 0
    ) * 100 * 1.1
    
    # Process each county for dashboard
    for i, county in enumerate(counties):
        # Row offset (title takes 2 rows, then each county takes 2 rows)
        row_offset = 2 + i * 2
        
        # Men subplot (left column) and women subplot (right column)
        men_ax = dashboard_fig.add_subplot(gs[row_offset, 0])
        women_ax = dashboard_fig.add_subplot(gs[row_offset, 1])
        
        # Age distribution subplot (row below, spans both columns)
        age_ax = dashboard_fig.add_subplot(gs[row_offset + 1, :])
        
        # Get all years with data for this county
        county_years = sorted(set(
            hajnal_men_df[hajnal_men_df['County'] == county]['Year'].unique().tolist() + 
            hajnal_women_df[hajnal_women_df['County'] == county]['Year'].unique().tolist() + 
            synthetic_df[synthetic_df['County'] == county]['Year'].unique().tolist()
        ))
        
        # Filter data for this county
        county_hajnal_men = hajnal_men_df[hajnal_men_df['County'] == county]
        county_synth_men = synthetic_df[(synthetic_df['County'] == county) & 
                                        (synthetic_df['Sex'] == 'Male')]
        
        county_hajnal_women = hajnal_women_df[hajnal_women_df['County'] == county]
        county_synth_women = synthetic_df[(synthetic_df['County'] == county) & 
                                          (synthetic_df['Sex'] == 'Female')]
        
        # Plot men's data in dashboard
        if not county_hajnal_men.empty:
            men_ax.plot(
                county_hajnal_men['Year'], 
                county_hajnal_men['Never_Married_Rate'] * 100,
                marker='o', linestyle='-', color=colors['hajnal_men'],
                linewidth=2, label="Hajnal Method"
            )
        
        if not county_synth_men.empty:
            men_ax.plot(
                county_synth_men['Year'], 
                county_synth_men['Synthetic_Never_Married_Rate'] * 100,
                marker='o', linestyle='--', color=colors['dynamic_men'],
                linewidth=2, label="Dynamic Method"
            )
        
        # Plot women's data in dashboard
        if not county_hajnal_women.empty:
            women_ax.plot(
                county_hajnal_women['Year'], 
                county_hajnal_women['Never_Married_Rate'] * 100,
                marker='s', linestyle='-', color=colors['hajnal_women'],
                linewidth=2, label="Hajnal Method"
            )
        
        if not county_synth_women.empty:
            women_ax.plot(
                county_synth_women['Year'], 
                county_synth_women['Synthetic_Never_Married_Rate'] * 100,
                marker='s', linestyle='--', color=colors['dynamic_women'],
                linewidth=2, label="Dynamic Method"
            )
        
        # Mark Copenhagen missing data in dashboard
        if county.lower() == 'københavn' and copenhagen_missing_year in county_years:
            for ax in [men_ax, women_ax]:
                ax.axvline(x=copenhagen_missing_year, color='red', linestyle='--', linewidth=2)
                ax.text(
                    copenhagen_missing_year + 0.5, 
                    max_rate * 0.95,
                    "A' (Missing Data)",
                    color='red', fontweight='bold', ha='left', va='top'
                )
        
        # Configure men and women plots in dashboard
        for ax, title in zip([men_ax, women_ax], 
                           [f'Men in {county.capitalize()}', 
                            f'Women in {county.capitalize()}']):
            ax.set_xlabel('Year', fontsize=12)
            ax.set_ylabel('Never-Married Rate (%)', fontsize=12)
            ax.set_title(title, fontsize=14)
            ax.set_ylim(0, max_rate)
            ax.set_xticks(county_years)
            ax.set_xticklabels(county_years, rotation=45)
            ax.grid(True, alpha=0.3)
            ax.legend(loc='best', fontsize=10)
            ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
        
        # Plot age distribution for this county (latest year) in dashboard
        county_dynamic_men = dynamic_men_df[dynamic_men_df['County'] == county]
        county_dynamic_women = dynamic_women_df[dynamic_women_df['County'] == county]
        
        if not county_dynamic_men.empty or not county_dynamic_women.empty:
            # Get latest year with data
            dynamic_years = sorted(set(
                county_dynamic_men['Year'].unique().tolist() + 
                county_dynamic_women['Year'].unique().tolist()
            ))
            
            if dynamic_years:
                latest_year = max(dynamic_years)
                
                # Plot age patterns for latest year
                for df, gender, color in [
                    (county_dynamic_men[county_dynamic_men['Year'] == latest_year], 
                     'Men', colors['hajnal_men']),
                    (county_dynamic_women[county_dynamic_women['Year'] == latest_year], 
                     'Women', colors['hajnal_women'])
                ]:
                    if not df.empty:
                        # Extract age groups and convert to midpoints
                        age_groups = df['Age_Group'].tolist()
                        age_midpoints = []
                        for ag in age_groups:
                            min_age, max_age = map(int, ag.split('-'))
                            age_midpoints.append((min_age + max_age) / 2)
                        
                        # Sort by age
                        idx = np.argsort(age_midpoints)
                        sorted_midpoints = [age_midpoints[k] for k in idx]
                        rates = df['Never_Married_Rate'].values[idx] * 100
                        
                        # Plot age pattern
                        age_ax.plot(
                            sorted_midpoints, rates,
                            marker='o' if gender == 'Men' else 's',
                            linestyle='-', color=color,
                            label=f"{gender} ({latest_year})"
                        )
                
                # Configure age plot in dashboard
                age_ax.set_xlabel('Age (midpoint of age group)', fontsize=12)
                age_ax.set_ylabel('Never-Married Rate (%)', fontsize=12)
                age_ax.set_title(f'Age-Specific Never-Married Rates in {county.capitalize()} ({latest_year})', 
                               fontsize=14)
                age_ax.grid(True, alpha=0.3)
                age_ax.legend(loc='best', fontsize=10)
                age_ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}%'))
                age_ax.set_ylim(0, 100)
    
    # Add summary box with key findings to dashboard
    if not hajnal_men_df.empty and not hajnal_women_df.empty:
        # Create summary subplot in the last row
        summary_ax = dashboard_fig.add_subplot(gs[rows-1, :])
        summary_ax.axis('off')
        
        # Find highest rates and gender gaps
        max_men_rate = hajnal_men_df.loc[hajnal_men_df['Never_Married_Rate'].idxmax()]
        max_women_rate = hajnal_women_df.loc[hajnal_women_df['Never_Married_Rate'].idxmax()]
        
        # Calculate gender gaps
        merged_data = pd.merge(
            hajnal_men_df[['County', 'Year', 'Never_Married_Rate']],
            hajnal_women_df[['County', 'Year', 'Never_Married_Rate']],
            on=['County', 'Year'],
            suffixes=('_men', '_women')
        )
        
        merged_data['gender_gap'] = merged_data['Never_Married_Rate_men'] - merged_data['Never_Married_Rate_women']
        max_gap = merged_data.loc[merged_data['gender_gap'].abs().idxmax()]
        
        # Create summary text
        findings = (
            f"Key Findings:\n"
            f"• Highest never-married rate for men: {max_men_rate['County']} ({max_men_rate['Year']}): {max_men_rate['Never_Married_Rate']*100:.1f}%\n"
            f"• Highest never-married rate for women: {max_women_rate['County']} ({max_women_rate['Year']}): {max_women_rate['Never_Married_Rate']*100:.1f}%\n"
            f"• Largest gender gap: {max_gap['County']} ({max_gap['Year']}): {abs(max_gap['gender_gap']*100):.1f}% "
            f"({'men' if max_gap['gender_gap'] > 0 else 'women'} more likely to remain unmarried)\n"
        )
        
        # Add visualization guide
        method_legend = (
            "Visualization Guide:\n"
            "• Solid lines with dark colors: Hajnal method (ages 49-54)\n"
            "• Dashed lines with lighter colors: Dynamic synthetic cohort method\n"
            f"• Red vertical line with 'A'' in København graphs: Missing data for year {copenhagen_missing_year}\n"
            "• All county graphs use the same scale for direct comparison"
        )
        
        # Add text box with findings to dashboard
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
        summary_ax.text(0.5, 0.5, findings + "\n\n" + method_legend, 
                      transform=summary_ax.transAxes, fontsize=12,
                      verticalalignment='center', horizontalalignment='center', 
                      bbox=props, linespacing=1.5)
    
    # Adjust layout and save dashboard
    plt.tight_layout()
    dashboard_fig.subplots_adjust(hspace=0.5, wspace=0.3)
    dashboard_path = 'never_married_dashboard.png'
    plt.savefig(dashboard_path, dpi=300, bbox_inches='tight')
    output_files['dashboard'] = dashboard_path
    plt.close(dashboard_fig)
    
    # Create comprehensive tables
    print("Creating comprehensive tables of never-married rates...")
    
    # Create a combined dataset for tables
    table_results = []
    
    # Process Hajnal data
    for df, gender in [(hajnal_men_df, 'Male'), (hajnal_women_df, 'Female')]:
        for _, row in df.iterrows():
            table_results.append({
                'County': row['County'],
                'Year': row['Year'],
                'Gender': gender,
                'Method': 'Hajnal',
                'Never_Married_Rate': row['Never_Married_Rate'] * 100,
                'Sample_Size': row['Sample_Size']
            })
    
    # Process Synthetic data
    for _, row in synthetic_df.iterrows():
        table_results.append({
            'County': row['County'],
            'Year': row['Year'],
            'Gender': row['Sex'],
            'Method': 'Dynamic',
            'Never_Married_Rate': row['Synthetic_Never_Married_Rate'] * 100,
            'Sample_Size': 'N/A'
        })
    
    # Convert to DataFrame and format
    table_df = pd.DataFrame(table_results)
    table_df = table_df.sort_values(['County', 'Year', 'Gender', 'Method'])
    table_df['Never_Married_Rate'] = table_df['Never_Married_Rate'].round(1)
    
    # Create pivot table for PDF
    pivot_df = table_df.pivot_table(
        index=['County', 'Year'],
        columns=['Gender', 'Method'],
        values='Never_Married_Rate',
        aggfunc='first'
    ).reset_index()
    
    # Sort by county and year
    pivot_df = pivot_df.sort_values(['County', 'Year'])
    
    # Save as CSV regardless of PDF availability
    csv_path = 'never_married_rates.csv'
    pivot_df.to_csv(csv_path, index=False)
    output_files['tables_csv'] = csv_path
    
    # Try to create PDF table
    try:
        pdf_path = 'never_married_rates.pdf'
        with PdfPages(pdf_path) as pdf:
            # Title page
            plt.figure(figsize=(8.5, 11))
            plt.axis('off')
            plt.text(0.5, 0.5, "Never-Married Rates in 19th Century Denmark\nBy County, Year, Gender, and Method",
                   ha='center', va='center', fontsize=16)
            pdf.savefig()
            plt.close()
            
            # Create a table for each county
            for county in pivot_df['County'].unique():
                county_data = pivot_df[pivot_df['County'] == county].drop(columns=['County'])
                
                fig, ax = plt.subplots(figsize=(8.5, 11))
                ax.axis('off')
                
                # Add county as title
                ax.text(0.5, 0.95, f"Never-Married Rates in {county.capitalize()}",
                       ha='center', va='top', fontsize=14, fontweight='bold')
                
                # Format the header to separate gender and method
                header = []
                for col in county_data.columns:
                    if isinstance(col, tuple):
                        header.append(f"{col[0]}\n({col[1]})")
                    else:
                        header.append(str(col))
                
                # Create the table
                table = ax.table(
                    cellText=county_data.values.tolist(),
                    colLabels=header,
                    loc='center',
                    cellLoc='center'
                )
                
                # Style the table
                table.auto_set_font_size(False)
                table.set_fontsize(9)
                table.scale(1.2, 1.5)
                
                # Style header and add percentage signs
                for (i, j), cell in table.get_celld().items():
                    if i == 0:  # Header row
                        cell.set_text_props(fontweight='bold')
                        cell.set_facecolor('#D3D3D3')
                    
                    # Add % to rate values
                    if i > 0 and j > 0:  # Data cells (not year)
                        try:
                            value = cell.get_text().get_text()
                            if value != 'nan':
                                cell.get_text().set_text(f"{value}%")
                        except:
                            pass
                
                pdf.savefig(fig, bbox_inches='tight')
                plt.close()
        
        output_files['tables_pdf'] = pdf_path
        print(f"Tables saved to PDF: {pdf_path}")
    except Exception as e:
        print(f"Error creating PDF: {e}. Data saved only as CSV.")
    
    # Save processed data for further analysis
    print("\nSaving processed data for further analysis...")
    for key, df in results.items():
        if not df.empty:
            csv_file = f"{key}_results.csv"
            df.to_csv(csv_file, index=False)
            output_files[f'{key}_csv'] = csv_file
    
    print("\nAnalysis complete! All visualizations and tables have been generated.")
    
    # Return both analysis results and output files
    return {
        'results': results,
        'output_files': output_files
    }

In [29]:
run_never_married_analysis(df_list=df_list, county_names=county_names)

Starting analysis of never-marriage rates in 19th century Denmark...
Processing census data and calculating never-married rates...

København County:
----------------------------------------
Men's never-married rates (Hajnal method):
  1787: 15.6% (n=3547)
  1801: 18.0% (n=4225)
  1834: 17.7% (n=1719)
  1840: 17.1% (n=3555)
  1845: 16.9% (n=3921)
  1850: 18.0% (n=4736)
  1860: 14.7% (n=5034)
  1880: 12.8% (n=7510)
  1901: 11.6% (n=13383)

Women's never-married rates (Hajnal method):
  1787: 10.6% (n=3467)
  1801: 12.5% (n=4210)
  1834: 13.3% (n=1875)
  1840: 15.9% (n=4516)
  1845: 16.6% (n=4699)
  1850: 18.2% (n=5456)
  1860: 19.0% (n=5865)
  1880: 18.3% (n=8798)
  1901: 21.5% (n=15949)

Århus County:
----------------------------------------
Men's never-married rates (Hajnal method):
  1787: 8.2% (n=745)
  1801: 8.0% (n=799)
  1834: 7.9% (n=882)
  1840: 6.0% (n=912)
  1845: 6.9% (n=1006)
  1850: 7.2% (n=1133)
  1860: 7.8% (n=1222)
  1880: 8.9% (n=1802)
  1901: 8.2% (n=2466)

Women's ne

/var/folders/xr/n973hf0s243g7hdvvvh5kzmr0000gn/T/ipykernel_17145/4288825935.py:563: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  county_data = pivot_df[pivot_df['County'] == county].drop(columns=['County'])


{'results': {'hajnal_men':        County  Year   Sex  Never_Married_Rate  Sample_Size  Rate_Percent
  0   københavn  1787  Male            0.156470         3547          15.6
  1   københavn  1801  Male            0.179882         4225          18.0
  2   københavn  1834  Male            0.177429         1719          17.7
  3   københavn  1840  Male            0.171308         3555          17.1
  4   københavn  1845  Male            0.168834         3921          16.9
  5   københavn  1850  Male            0.180321         4736          18.0
  6   københavn  1860  Male            0.147000         5034          14.7
  7   københavn  1880  Male            0.127963         7510          12.8
  8   københavn  1901  Male            0.116117        13383          11.6
  9       århus  1787  Male            0.081879          745           8.2
  10      århus  1801  Male            0.080100          799           8.0
  11      århus  1834  Male            0.079365          882           7.9
